In [1]:
! pip install spacy
! pip install spacy-transformers
! python -m spacy download en_core_web_trf
! pip install pytextrank
! pip install vaderSentiment

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 8.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [2]:
import spacy
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf 
import datasets
import re
from spacy.language import Language
from spacy.tokens import Span

2024-06-06 16:47:30.590152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 16:47:32.375389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


text-classification

In [3]:
nlp = spacy.load('en_core_web_trf')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
with open ("/teamspace/studios/this_studio/text-comperiahand/data/text1.txt", "r") as f:
    text = f.read()

In [5]:
doc = nlp(text)

In [6]:
print(doc)

I enjoyed visiting the spa. It was very comfortable but it was also very expensive. The amenities were ok but the service made the spa a great experience



In [7]:
import spacy
from spacy import displacy
import re 

# Load the transformer-based spaCy model
nlp = spacy.load('en_core_web_trf')

EMAIL_PATTERN = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

@Language.component("email_component")
def email_component(doc):
    # Find matches in the text
    matches = EMAIL_PATTERN.finditer(doc.text)
    # Create spans from the matches
    spans = [Span(doc, doc.char_span(match.start(), match.end()).start, doc.char_span(match.start(), match.end()).end, label="EMAIL") for match in matches if doc.char_span(match.start(), match.end())]
    # Merge spans with existing entities
    doc.ents = list(doc.ents) + spans
    return doc



# Define a new text with entities
new_text = """Hello Mr.strenger, I am vinayak. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0008 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account number XXXXXX1111 with the routing number XXXXX0000.
Customer feedback for Sunshine Spa, 123 Main St, Anywhere. Send comments to Alice at sunspa@mail.com. 
I enjoyed visiting the spa. It was very comfortable but it was also very expensive. The amenities were ok but the service made the spa a great experience."""
nlp.add_pipe('email_component', after='ner')

# Process the new text
doc = nlp(new_text)



# Print pipeline components
#print("Pipeline components:", nlp.pipe_names)

# Check if NER component is in the pipeline
if 'ner' not in nlp.pipe_names:
    print("NER component is missing from the pipeline.")
else:
   print("NER component is present.")

# Print named entities
print("Named Entities in the text:")
for ent in doc.ents:
    print(ent.text, ent.label_)


print(displacy.render(doc, style="ent"))




NER component is present.
Named Entities in the text:
Mr.strenger PERSON
vinayak PERSON
AnyCompany Financial Services, LLC ORG
24.53 MONEY
July 31st DATE
Sunshine Spa ORG
Alice PERSON
sunspa@mail.com EMAIL


None


In [8]:
nlp.analyze_pipes()

{'summary': {'transformer': {'assigns': ['doc._.trf_data'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'email_component': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'tra

#key-phrase confidance

In [9]:
import spacy
import pytextrank

# Load the spaCy model
nlp = spacy.load('en_core_web_trf')

# Add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

# Define the text
text = """Hello Mr.strenger, I am vinayak. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0008 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account number XXXXXX1111 with the routing number XXXXX0000.
Customer feedback for Sunshine Spa, 123 Main St, Anywhere. Send comments to Alice at sunspa@mail.com.
I enjoyed visiting the spa. It was very comfortable but it was also very expensive. The amenities were ok but the service made the spa a great experience."""

# Process the text
doc = nlp(text)

# Extract and print key phrases with their confidence scores
print("Top Document Keywords:")
for phrase in doc._.phrases[:10]:
    print(f"Key Phrase: {phrase.text}, Confidence: {phrase.rank:.4f}, Count: {phrase.count}")



/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages
Top Document Keywords:
Key Phrase: Sunshine Spa, Confidence: 0.1137, Count: 2
Key Phrase: July 31st, Confidence: 0.0848, Count: 2
Key Phrase: Your AnyCompany Financial Services, LLC credit card account, Confidence: 0.0791, Count: 1
Key Phrase: Alice, Confidence: 0.0762, Count: 2
Key Phrase: Mr.strenger, Confidence: 0.0762, Count: 1
Key Phrase: comments, Confidence: 0.0762, Count: 1
Key Phrase: vinayak, Confidence: 0.0762, Count: 2
Key Phrase: AnyCompany Financial Services, LLC, Confidence: 0.0755, Count: 1
Key Phrase: your bank account number, Confidence: 0.0751, Count: 1
Key Phrase: the due date, Confidence: 0.0654, Count: 1


language detect 

In [10]:
! pip install langdetect


In [11]:
from langdetect import detect

# Define the text
text = "एक या एक से अधिक"

# Detect the language of the text
language = detect(text)

# Print the detected language
print("Detected Language:", language)


Detected Language: hi


Personally identifiable information (PII) analysis mode

In [12]:
import spacy
from spacy.tokens import Doc
from spacy.matcher import Matcher
import re

# Load the transformer-based spaCy model
nlp = spacy.load("en_core_web_trf")

# Define regular expressions for PII entities
EMAIL_PATTERN = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
PHONE_NUMBER_PATTERN = re.compile(r'\b(?:\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b')

# Define custom components for PII entity recognition
@spacy.Language.component("pii_email_component")
def pii_email_component(doc):
    matches = EMAIL_PATTERN.finditer(doc.text)
    spans = []
    for match in matches:
        start, end = match.start(), match.end()
        span = doc.char_span(start, end, label="EMAIL")
        if span is not None:
            spans.append(span)
    doc.ents = list(doc.ents) + spans
    return doc

@spacy.Language.component("pii_phone_component")
def pii_phone_component(doc):
    matches = PHONE_NUMBER_PATTERN.finditer(doc.text)
    spans = []
    for match in matches:
        start, end = match.start(), match.end()
        span = doc.char_span(start, end, label="PHONE_NUMBER")
        if span is not None:
            spans.append(span)
    doc.ents = list(doc.ents) + spans
    return doc

# Add custom components to the spaCy pipeline
nlp.add_pipe("pii_email_component", after="ner")
nlp.add_pipe("pii_phone_component", after="pii_email_component")

# Process text and extract PII entities
text = """Hello Mr.strenger, I am vinayak. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0008 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account number XXXXXX1111 with the routing number XXXXX0000.
Customer feedback for Sunshine Spa, 123 Main St, Anywhere. Send comments to Alice at sunspa@mail.com.
I enjoyed visiting the spa. It was very comfortable but it was also very expensive. The amenities were ok but the service made the spa a great experience."""

doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)


Mr.strenger PERSON
vinayak PERSON
AnyCompany Financial Services, LLC ORG
24.53 MONEY
July 31st DATE
Sunshine Spa ORG
Alice PERSON
sunspa@mail.com EMAIL


tagged sentiment 

In [15]:
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from spacy.tokens import Token

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the VADER sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Register custom extension attribute "sentiment"
if not Token.has_extension("sentiment"):
    Token.set_extension("sentiment", default="")

# Function to tag sentiment in text
def tag_sentiment(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Analyze sentiment for each sentence
    for sentence in doc.sents:
        # Get the sentiment score for the sentence using VADER
        sentiment_score = sentiment_analyzer.polarity_scores(sentence.text)
        # Tag sentiment based on the compound score
        if sentiment_score['compound'] >= 0.05:
            sentiment_tag = 'Positive'
        elif sentiment_score['compound'] <= -0.05:
            sentiment_tag = 'Negative'
        else:
            sentiment_tag = 'Neutral'
        # Add sentiment tag to each token in the sentence
        for token in sentence:
            token._.sentiment = sentiment_tag
    return doc

# Example text
text = "I love this product. It's amazing! But the customer service was terrible."

# Tag sentiment in the text
tagged_doc = tag_sentiment(text)

# Print sentiment tags for each token
for token in tagged_doc:
    print(token.text, token._.sentiment)


I Positive
love Positive
this Positive
product Positive
. Positive
It Positive
's Positive
amazing Positive
! Positive
But Negative
the Negative
customer Negative
service Negative
was Negative
terrible Negative
. Negative


syntex

In [17]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_trf")

# Process text
text = "SpaCy is an awesome library for natural language processing."
doc = nlp(text)

# Analyze syntax
for token in doc:
    print(token.text, token.pos_)


SpaCy PROPN
is AUX
an DET
awesome ADJ
library NOUN
for ADP
natural ADJ
language NOUN
processing NOUN
. PUNCT
